## Analyzing CIA Factbook Data on Countries Using SQL

The [CIA factbook](https://www.cia.gov/the-world-factbook/) is a compendium of statistics about all of the countries on Earth. 
It contains information such as:
- **name** — the name of the country.
- **area** — the country's total area (both land and water).
- **area_land** — the country's land area in square kilometers.
- **area_water** — the country's waterarea in square kilometers.
- **population** — the country's population.
- **population_growth** — the country's population growth as a percentage.
- **birth_rate** — the country's birth rate, or the number of births per year per 1,000 people.
- **death_rate** — the country's death rate, or the number of death per year per 1,000 people.

In this project, we will extract some insights from this SQLite [database](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db) using SQL.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [2]:
%%sql

SELECT *
    FROM sqlite_master
    WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql

SELECT *
    FROM facts
    LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


We'll start by calculating some summary statistics and look for any outlier countries. 

## Exploring Population

We'll begin by retrieving the countries with the smallest and largest `populations`.

In [4]:
%%sql

SELECT MIN(population) as min_pop,
    MAX(population) as max_pop
    FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop
0,7256490011


Right away, we can see that there seems to be a country with **0 inhabitants** and one with more than **7 billions**. Let's find out which ones they are.

In [5]:
%%sql

/*Selecting the country with the minimum population*/

SELECT *
    FROM facts
    WHERE population == (SELECT MIN(population)
                         FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [6]:
%%sql

/*Selecting the country with the maximum population*/

SELECT *
    FROM facts
    WHERE population == (SELECT MAX(population)
                         FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


It appears `Antartica` is being listed as the country with 0 population, while `World` appears as the row with over 7 billion citizens. Since neither of them is an actual country, we'll exclude them and repeat the previous analysis.

In [7]:
%%sql

SELECT MIN(population) as min_pop,
    MAX(population) as max_pop
    FROM facts
    WHERE population != 0
    AND population != 7256490011

 * sqlite:///factbook.db
Done.


min_pop,max_pop
48,1367485388


In [8]:
%%sql

/*Selecting the countries with the minimun population*/

SELECT name, population
    FROM facts
    WHERE population == (SELECT MIN(population)
                             FROM facts
                            WHERE population != 0);

 * sqlite:///factbook.db
Done.


name,population
Pitcairn Islands,48


In [9]:
%%sql

/*Selecting the countries with the maximum population*/

SELECT name, population
    FROM facts
    WHERE population == (SELECT MAX(population)
                             FROM facts
                            WHERE population != 7256490011);

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


We can now see that the world's most populated country is `China`, with over **1.3 billion inhabitants**, while the `Pitcairn Islands` are the least populated nation, with only **48 people** living there.

## Finding Densely Populated Countries

Let's explore the **average** values for `population` and `area`.

In [10]:
%%sql

SELECT AVG(population) AS avg_pop, 
    AVG(area) AS avg_area 
    FROM facts
    WHERE name != 'World';

 * sqlite:///factbook.db
Done.


avg_pop,avg_area
32242666.56846473,555093.546184739


We can see that the average `population` is around 32 millions and the average `area` is 555 thousand square kilometers.


We'll now determine which countries are the **most densely populated**. To do this, we'll identify countries that present:
- Above-average values for `population`.
- Below-average values for `area`.

In [11]:
%%sql

SELECT *
    FROM facts 
    WHERE population > (SELECT AVG(population)
                        FROM facts)
    AND area < (SELECT AVG(area)
                        FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


Now let's find the **least densely populated** countries. These would have:
- Below-average values for `population`.
- Above-average values for `area`.

In [12]:
%%sql

SELECT *
    FROM facts 
    WHERE population < (SELECT AVG(population)
                        FROM facts)
    AND area > (SELECT AVG(area)
                        FROM facts);

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.0
9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
21,bl,Bolivia,1098581,1083301,15280,10800882,1.56,22.76,6.52,0.62
23,bc,Botswana,581730,566730,15000,2182719,1.21,20.96,13.39,4.56
28,bm,Burma,676578,653508,23070,56320206,1.01,18.39,7.96,0.28
32,ca,Canada,9984670,9093507,891163,35099836,0.75,10.28,8.42,5.66
34,ct,Central African Republic,622984,622984,0,5391539,2.13,35.08,13.8,0.0


To be continued:
- Which country has the highest growth rate?
- Which countries have the highest ratios of water to land? Which countries have more water than land?
- Which countries will add the most people to their populations next year?
- Which countries have a higher death rate than birth rate?
- Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?